In [1]:
# imports
# from transformers import pipeline
from helpers import Video_loader, Image_loader, Captioner

In [2]:
VIDEOS_DIRECTORY = "videos"
FRAME_FREQUENCY = 6000

In [3]:
# video captions section
video_loader = Video_loader(VIDEOS_DIRECTORY, FRAME_FREQUENCY)

captions_dict = video_loader.process()
# need to tune this up but for now it's whatever - we have frame visual captions now

Init video loader
Make frames
Storing in videos\Why the new MacBook Air is a BIG deal.mp4_images
Start image loading
Captioning subfolder: videos\Why the new MacBook Air is a BIG deal.mp4_images
Init image loader
Init captioner


All model checkpoint layers were used when initializing TFBlipForConditionalGeneration.

All the layers of TFBlipForConditionalGeneration were initialized from the model checkpoint at Salesforce/blip-image-captioning-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBlipForConditionalGeneration for predictions without further training.


Start image get captions
Get caption


c:\Users\david\anaconda3\envs\research-env\lib\site-packages\transformers\generation\tf_utils.py:854: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'a man with glasses and a beard leaning on a yellow bench'}]
Get caption
[{'generated_text': 'a man sitting at a table with a laptop'}]
Get caption
[{'generated_text': 'a bunch of different screens and screens'}]
[[{'generated_text': 'a man with glasses and a beard leaning on a yellow bench'}], [{'generated_text': 'a man sitting at a table with a laptop'}], [{'generated_text': 'a bunch of different screens and screens'}]]


In [ ]:
# audio captions section
